# Exercise 4

## 1. Constructing the matrix X

In the first step we are going to implemnt the function X = construct_X(M, alphas, Np = None) without any sparse matrix optimizations

In [ ]:
def construct_X(M, alphas, Np = None):
    



    return X


## 2. Recovering the image

## 3. Minimizing the radiation dose